**Step 1: Import libraries**

In [1]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import cv2

**Step 2: Load Model**

In [3]:
interpreter = tf.lite.Interpreter(model_path='lite-model_movenet_singlepose_lightning_3.tflite')
interpreter.allocate_tensors()

**Step 3: Make Detection**

**Function: Draw Keypoints & Connections**

In [ ]:
# Function to draw keypoints and connections
def draw_keypoints_and_connections(image, keypoints_with_scores, threshold=0.4):
    height, width, _ = image.shape
    keypoints = keypoints_with_scores[0][0]
    
    # Define the edges to connect keypoints (similar to MediaPipe)
    EDGES = {
        (0, 1): 'm', (0, 2): 'm', (1, 3): 'c', (2, 4): 'c',
        (0, 5): 'm', (0, 6): 'm', (5, 7): 'y', (7, 9): 'y',
        (6, 8): 'y', (8, 10): 'y', (5, 6): 'm', (5, 11): 'm',
        (6, 12): 'm', (11, 13): 'm', (13, 15): 'y', (12, 14): 'y',
        (14, 16): 'y', (11, 12): 'm'
    }
    
    # Draw keypoints
    for idx, keypoint in enumerate(keypoints):
        y, x, confidence = keypoint
        if confidence > threshold:
            cv2.circle(image, (int(x * width), int(y * height)), 5, (0, 255, 0), -1)

    # Draw connections
    for (p1, p2), color in EDGES.items():
        y1, x1, c1 = keypoints[p1]
        y2, x2, c2 = keypoints[p2]
        if c1 > threshold and c2 > threshold:
            cv2.line(image, (int(x1 * width), int(y1 * height)), 
                     (int(x2 * width), int(y2 * height)), (0, 255, 0), 2)

In [ ]:
# Start video capture
cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # Preprocess image
    img = tf.image.resize_with_pad(np.expand_dims(frame, axis=0), 192, 192)
    input_image = tf.cast(img, dtype=tf.float32)
    
    # Setup input and output for the interpreter
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    
    # Set input tensor
    interpreter.set_tensor(input_details[0]['index'], np.array(input_image))
    interpreter.invoke()
    
    # Get output keypoints with scores
    keypoints_with_scores = interpreter.get_tensor(output_details[0]['index'])
    
    # Draw keypoints and connections
    draw_keypoints_and_connections(frame, keypoints_with_scores, threshold=0.4)
    
    # Display the frame
    cv2.imshow('MoveNet Lightning', frame)
    
    # Exit screen if ESC or q button was pressed
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
    if(cv2.waitKey(1) & 0xFF == 27):
        break

cap.release()
cv2.destroyAllWindows()